In [1]:
import scanpy as sc
import os
import pandas as pd
import numpy as np

import os, sys
from pathlib import Path

script_dir = Path().resolve()
repo_dir = script_dir
src_dir = repo_dir / 'src'
data_dir = repo_dir / 'data'
sys.path.append(str(src_dir))
sys.path.append(str(data_dir))

from dataset.dataloader import AnnDataDataset

PARAMETERS = {
    # 'hvgs': 20116,
    # 'num_genes': 20116,
    # 'hvgs': 5000,
    # 'num_genes': 5000,
    'latent_dimension': 50,
    'target_sum': 10000,
    'batch_size': 128,
    'num_epochs': 1,
}

In [ ]:
# ts_data = sc.read(
#     r'./data/raw/tabula_sapiens_v2/tabula_sapiens.h5ad',
# )

In [ ]:
print('skipped')
# sc.pp.calculate_qc_metrics(
#     ts_data,
#     qc_vars=["mt"],        # v2 uses standard 'mt' gene prefix for mitochondrials
#     var_type="gene",
#     inplace=True,
# )

In [ ]:
immune_only = ts_data[ts_data.obs['compartment']=='Immune']

In [ ]:
immune_only

In [ ]:
immune_only.obs['sex']

In [ ]:
immune_female_adata = immune_only[immune_only.obs['sex']=='female']

In [ ]:
immune_female_adata

In [ ]:
import numpy as np
x = immune_female_adata.X
if isinstance(x, np.ndarray):         # dense
    mx, mean_gene_var = x.max(), np.var(x, axis=0).mean()
else:                                 # sparse
    mx, mean_gene_var = x.max(), np.var(x.A, axis=0).mean()

print(f"max value {mx:0.1f}  • mean gene variance {mean_gene_var:0.2f}")

In [ ]:
import pickle
with open(r'./data/pickled/tabula_muris/ts_adata_immune_female_61759_features.pkl', 'wb') as f:
    pickle.dump(immune_female_adata, f)

In [ ]:
pc = immune_female_adata.var['feature_type'] == 'protein_coding'
immune_female_pc_adata = immune_female_adata[:, pc]
immune_female_pc_adata

In [ ]:
# with open(r'./data/pickled/tabula_muris/ts_adata_immune_female_20001_protein_coding.pkl', 'wb') as f:
#     pickle.dump(immune_female_pc_adata, f)

In [2]:
import pickle
with open(r'./data/pickled/tabula_sapiens_v2/ts_adata_immune_female_20001_protein_coding.pkl', 'rb') as f:
    immune_female_pc_adata=pickle.load(f)

In [ ]:
# immune_female_pc_adata.obs['cell_type']
immune_female_pc_adata.obs.rename(columns={'cell_type': 'Celltype'}, inplace=True)



In [8]:
with open(r'./data/pickled/tabula_sapiens_v2/ts_adata_immune_female_20001_protein_coding.pkl', 'wb') as f:
    pickle.dump(immune_female_pc_adata, f)

In [10]:
immune_female_pc_adata

View of AnnData object with n_obs × n_vars = 326286 × 20001
    obs: 'donor_id', 'tissue_in_publication', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'assay_ontology_term_id', 'sample_id', 'replicate', '10X_run', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_type_ontology_term_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'ethnicity_original', 'sample_number', 'organism_ontology_term_id', 'suspension_type', 'tissue_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'Celltype', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_

In [11]:
sc.pp.highly_variable_genes(
    immune_female_pc_adata,
    batch_key="method",
)

/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:696: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["hvg"] = {"flavor": flavor}


In [12]:
immune_female_pc_adata

AnnData object with n_obs × n_vars = 326286 × 20001
    obs: 'donor_id', 'tissue_in_publication', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'assay_ontology_term_id', 'sample_id', 'replicate', '10X_run', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_type_ontology_term_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'ethnicity_original', 'sample_number', 'organism_ontology_term_id', 'suspension_type', 'tissue_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'Celltype', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicit

In [13]:
from torch.utils.data import DataLoader

ts_dataset = AnnDataDataset(immune_female_pc_adata)
ts_dataloader = DataLoader(ts_dataset, batch_size=2048, shuffle=True)

/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:459: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: invalid value encountered in divide
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:461: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_g

/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:459: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:458: RuntimeWarning: invalid value encountered in divide
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:461: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/oscar/data/rsingh47/wli115/scGrapHiCv2/pytorch.venv/lib64/python3.9/site-packages/scanpy/tools/_rank_genes_g

In [14]:
import pickle
with open(r'./data/pickled/tabula_sapiens_v2/tm_dataset_immune_female_20001_protein_coding.pkl', 'wb') as f:
    pickle.dump(ts_dataset, f)

with open(r'./data/pickled/tabula_sapiens_v2/tm_dataloader_immune_female_20001_protein_coding.pkl', 'wb') as f:
    pickle.dump(ts_dataloader, f)

In [15]:
import warnings
import torch
# Define functions to precompute data-dependent variables
def precompute_gene_clusters(dataset):
    most_significant_genes_dict = dataset.most_significant_genes_dict
    least_significant_genes_dict = dataset.least_significant_genes_dict
    gene_networks = dataset.gene_networks
    cell_type_categories = dataset.cell_type_categories
    code_to_celltype = dataset.code_to_celltype
    celltype_to_code = dataset.celltype_to_code
    gene_names = dataset.gene_names
    gene_name_to_index = dataset.gene_name_to_index
    index_to_gene_name = dataset.index_to_gene_name
    gene_dispersions = dataset.gene_dispersions
    print('Precomputed gene clusters!')
    return (most_significant_genes_dict, least_significant_genes_dict,
            gene_networks, gene_names, code_to_celltype, celltype_to_code,
            gene_name_to_index, index_to_gene_name, gene_dispersions)

def precompute_mu_sigma(dataloader, most_significant_genes_dict, least_significant_genes_dict, gene_name_to_index):
    all_expression_matrix = []
    cell_types_data = {}
    cell_types_msg_data = {}
    cell_types_lsg_data = {}
    for batch in dataloader:
        expression_matrix, cell_types = batch
        all_expression_matrix.append(expression_matrix)
        
        for cell_type in torch.unique(cell_types):
            cell_type = int(cell_type)
            cell_type_mask = cell_types == cell_type
            cell_type_expression_matrix = expression_matrix[cell_type_mask]
            # All genes
            if cell_type not in cell_types_data:
                cell_types_data[cell_type] = []
            cell_types_data[cell_type].append(cell_type_expression_matrix)

            # Most significant genes
            # msg_genes = most_significant_genes_dict[cell_type]
            # missing = [g for g in msg_genes if g not in gene_name_to_index]
            # print(f"{len(missing)} / {len(msg_genes)} genes missing:", missing[:10])

            # msg_gene_indices = [gene_name_to_index[g] for g in msg_genes]
            msg_genes = most_significant_genes_dict[cell_type]
            msg_gene_indices = [
                gene_name_to_index[g]          # lookup
                for g in msg_genes
                if g in gene_name_to_index     # keep only genes that exist
            ]
            if not msg_gene_indices:
                warnings.warn(f"No MSG genes found for cell type {cell_type}. "
                                "Check identifier mismatches.")

            msg_significant_gene_matrix = cell_type_expression_matrix[:, msg_gene_indices]
            if cell_type not in cell_types_msg_data:
                cell_types_msg_data[cell_type] = []
            cell_types_msg_data[cell_type].append(msg_significant_gene_matrix)
            
            # Least significant genes
            # lsg_gene_indices = [gene_name_to_index[g] for g in lsg_genes]
            lsg_genes = least_significant_genes_dict[cell_type]
            lsg_gene_indices = [
                gene_name_to_index[g]          # lookup
                for g in lsg_genes
                if g in gene_name_to_index     # keep only genes that exist
            ]
            if not lsg_gene_indices:
                warnings.warn(f"No LSG genes found for cell type {cell_type}. "
                                "Check identifier mismatches.")

            lsg_significant_gene_matrix = cell_type_expression_matrix[:, lsg_gene_indices]
            if cell_type not in cell_types_lsg_data:
                cell_types_lsg_data[cell_type] = []
            cell_types_lsg_data[cell_type].append(lsg_significant_gene_matrix)

    cell_type_mu_sigma = {}
    cell_type_msg_mu_sigma = {}
    cell_type_lsg_mu_sigma = {}
    # All genes
    for cell_type, cell_type_expression_matrix in cell_types_data.items():
        data_tensor = torch.cat(cell_type_expression_matrix, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        cell_type_mu_sigma[int(cell_type)] = (mu, sigma)
    
    # Most significant genes
    for cell_type, matrices in cell_types_msg_data.items():
        data_tensor = torch.cat(matrices, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        dispersion = sigma**2 / mu
        cell_type_msg_mu_sigma[int(cell_type)] = (mu, sigma, dispersion)
    
    # Least significant genes
    for cell_type, matrices in cell_types_lsg_data.items():
        data_tensor = torch.cat(matrices, dim=0)
        mu = torch.mean(data_tensor, dim=0)
        sigma = torch.std(data_tensor, dim=0, unbiased=False)
        sigma = torch.clamp(sigma, min=1e-8)
        dispersion = sigma**2 / mu
        cell_type_lsg_mu_sigma[int(cell_type)] = (mu, sigma, dispersion)

    data_tensor = torch.cat(all_expression_matrix, dim=0)
    global_mu_sigma = (torch.mean(data_tensor, dim=0),
                       torch.std(data_tensor, dim=0, unbiased=False))

    return cell_type_mu_sigma, global_mu_sigma, cell_type_msg_mu_sigma, cell_type_lsg_mu_sigma

# Precompute data-dependent variables before model initialization
(most_significant_genes_dict, least_significant_genes_dict,
 gene_networks, gene_names, code_to_celltype, celltype_to_code,
 gene_name_to_index, index_to_gene_name, gene_dispersions) = precompute_gene_clusters(ts_dataset)

cell_type_mu_sigma, global_mu_sigma, cell_type_msg_mu_sigma, cell_type_lsg_mu_sigma = precompute_mu_sigma(
    ts_dataloader, most_significant_genes_dict, least_significant_genes_dict, gene_name_to_index)

Precomputed gene clusters!


/oscar/data/rsingh47/wli115/scContrast/src/dataset/dataloader.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_item = torch.tensor(self.y[idx], dtype=torch.long)  # Return integer-encoded cell type
/tmp/ipykernel_3186603/1284986444.py:51: UserWarning: No MSG genes found for cell type 18. Check identifier mismatches.
  warnings.warn(f"No MSG genes found for cell type {cell_type}. "
/tmp/ipykernel_3186603/1284986444.py:91: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  sigma = torch.std(data_tensor, dim=0, unbiased=False)


In [16]:
precomputed_dir = data_dir / 'pickled' / 'tabula_sapiens_v2' / 'precomputed'
precomputed_dir.mkdir(parents=True, exist_ok=True)

precomputed_gene_clusters_path =  precomputed_dir / 'ts_dataset_immune_female_20001_protein_coding_precomputed_gene_clusters.pkl'
with open(precomputed_gene_clusters_path, 'wb') as f:
    pickle.dump(
        {
            "most_significant_genes_dict": most_significant_genes_dict,
            "least_significant_genes_dict": least_significant_genes_dict,
            "gene_networks": gene_networks,
            "gene_names": gene_names,
            "code_to_celltype": code_to_celltype,
            "celltype_to_code": celltype_to_code,
            "gene_name_to_index": gene_name_to_index,
            "index_to_gene_name": index_to_gene_name,
            "gene_dispersions": gene_dispersions,
        },
        f,
    )

precomputed_mu_sigma_path = precomputed_dir / 'ts_dataset_immune_female_20001_protein_coding_precomputed_mu_sigma.pkl'
with open(precomputed_mu_sigma_path, "wb") as f:
    pickle.dump(
        {
            "cell_type_mu_sigma": cell_type_mu_sigma,
            "global_mu_sigma": global_mu_sigma,
            "cell_type_msg_mu_sigma": cell_type_msg_mu_sigma,
            "cell_type_lsg_mu_sigma": cell_type_lsg_mu_sigma,
        },
        f,
    )